Delta Task 2 Project WIP

In [1]:
import pandas as pd

In [4]:
tweets_test = pd.read_csv("btc_tweets_test.parquet.gzip")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x84 in position 7: invalid start byte

In [7]:
import pandas as pd
df = pd.read_parquet("btc_tweets_test.parquet.gzip")

In [10]:
df.head()

,hashtags,content,username,user_displayname,sentiment
tweet ID,,,,,
1641861708246552576,"[crypto, btc]",#crypto $crypto #btc \nI am Chinese crypto alp...,huahuayjy,花花研究院 | Crypto Alpha🇨🇳,True
1641861783898972167,"[Bitcoin, Bitcoin]",#Bitcoin would have to fall another 80% to rea...,luke_broyles,Luke Broyles,False
1641862152532418562,"[Giveaway, BTC, SolanaGiveaways, Giveaway, Air...",#Giveaway $1000 Matic in 3Days\n\n🏆To win\n1️⃣...,cryptomarsdo,Crypto Mars,True
1641862338369183753,"[EOS, USDT, BTC, crypto, Bitcoin, etherium, Bi...",Up or Down?\n\n!!! $EOS #EOS !!!\n\nVS\n\n$USD...,andreyukrnet,Andrey Ukraine,True
1641862430434131968,"[BTC, ETH, BSC, GroveToken]",Mid Day Mix-up is LIVE! Never know who might s...,JustAman04,Justin Anderson,True


In [9]:
df['content']

,content
tweet ID,
1641861708246552576,#crypto $crypto #btc \nI am Chinese crypto alp...
1641861783898972167,#Bitcoin would have to fall another 80% to rea...
1641862152532418562,#Giveaway $1000 Matic in 3Days\n\n🏆To win\n1️⃣...
1641862338369183753,Up or Down?\n\n!!! $EOS #EOS !!!\n\nVS\n\n$USD...
1641862430434131968,Mid Day Mix-up is LIVE! Never know who might s...
...,...
1641952735653421056,The US government puts a guy in jail for 2 lif...
1641952895112290305,#Bitcoin Embrace it ⚡️ ₿ ₿ ₿⚡️ https://t.co/lq...
1641952949763973122,☆☆☆ONWARD UPWARD BABE\ni love stamps i love or...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 1641861708246552576 to 1641953216999968769
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   hashtags          500 non-null    object
 1   content           500 non-null    object
 2   username          500 non-null    object
 3   user_displayname  500 non-null    object
 4   sentiment         500 non-null    bool  
dtypes: bool(1), object(4)
memory usage: 20.0+ KB
